In [1]:
# !pip install pandas
# !pip install numpy

In [2]:
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None # Fuck warnings

<h1> Загрузка данных

In [3]:
import json


with open(r'C:\Users\User\Documents\snli_1.0/snli_1.0_train.jsonl', 'r') as json_file:
    json_list = list(json_file)

result = []
for json_str in json_list:
    result.append(json.loads(json_str))

In [4]:
df_train = pd.json_normalize(result)

In [5]:
df_cut = df_train[:10000]

In [6]:
df_cut.columns

Index(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1',
       'sentence1_binary_parse', 'sentence1_parse', 'sentence2',
       'sentence2_binary_parse', 'sentence2_parse'],
      dtype='object')

In [7]:
df_cut.head(1)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...


<h1> Предобработка данных

<h3> Оставляем только основу от слов

In [22]:
import nltk.stem


my_stem = nltk.stem.SnowballStemmer('english')

def stemSentence(sentence):
    new_sentence = ''

    for word in sentence.split():
        new_sentence += my_stem.stem(word) + ' '
    
    return new_sentence[:-1]

<h3> Убираем символы

In [17]:
import string

def deleteSymbs(sentence):
    res = ''

    for char in sentence:
        if char not in string.punctuation:
            res += char
    
    return res

'Hello world'

<h3> Agregate function

In [24]:
def fixSentence(sentence):
    for func in ['deleteSymbs', 'stemSentence']:
        sentence = eval(func)(sentence)
    
    return sentence

In [26]:
for i in [1, 2]:
    df_cut[f'Fixed_sentence{i}'] = df_cut[f'sentence{i}'].apply(fixSentence)

<h1> Word2Vec

In [9]:
import gensim


texts = np.concatenate((df_cut.Stemmed_sentence1.values, df_cut.Stemmed_sentence1.values))

['a person on a hors jump over a broken down airplane. '
 'a person on a hors jump over a broken down airplane. '
 'a person on a hors jump over a broken down airplane. ' ...
 'a small babi in a white and purpl onesi cri while lay on a green blanket. '
 'a small babi in a white and purpl onesi cri while lay on a green blanket. '
 'babi in a lavend onesi lie on a pastel green blanket on a couch crying. ']


In [10]:
model = gensim.models.Word2Vec(
        texts,
        size=150,
        window=10,
        min_count=2,
        workers=10,
        iter=10)

In [16]:
model.wv.most_similar(positive='person')

KeyError: "word 'person' not in vocabulary"